In [3]:
import os
import time
import json
import requests
import pandas as pd
from datetime import datetime
from typing import List, Dict, Optional
from google.colab import userdata


# ============================================================
# CONFIGURATION & AUTHENTIFICATION
# ============================================================

CLIENT_ID = userdata.get("GBP_CLIENT_ID")
CLIENT_SECRET = userdata.get("GBP_CLIENT_SECRET")
REFRESH_TOKEN = userdata.get("GBP_REFRESH_TOKEN")


# Vérification proactive des identifiants
if not all([CLIENT_ID, CLIENT_SECRET, REFRESH_TOKEN]):
    missing_secrets = []
    if not CLIENT_ID: missing_secrets.append("GBP_CLIENT_ID")
    if not CLIENT_SECRET: missing_secrets.append("GBP_CLIENT_SECRET")
    if not REFRESH_TOKEN: missing_secrets.append("GBP_REFRESH_TOKEN")

    print("❌ ERREUR CRITIQUE : Identifiants manquants dans les Secrets Colab.")
    print(f"Secrets introuvables : {', '.join(missing_secrets)}")
    print("Action requise : Ajoutez-les via le menu 'Clé' (🔑) à gauche et cochez 'Accès au notebook'.")
    raise ValueError("Arrêt du script : Secrets API Google non configurés.")
else:
    print("✅ Identifiants chargés. Prêt pour l'authentification OAuth.")

✅ Identifiants chargés. Prêt pour l'authentification OAuth.


In [7]:
TOKEN_URL = "https://oauth2.googleapis.com/token"

# --- NOUVELLE ARCHITECTURE API (Fédérée) ---
# 1. Pour les comptes : Account Management API v1
API_ACCOUNTS_BASE = "https://mybusinessaccountmanagement.googleapis.com/v1"
# 2. Pour les établissements : Business Information API v1
API_LOCATIONS_BASE = "https://mybusinessbusinessinformation.googleapis.com/v1"
# 3. Pour les avis : My Business API v4 (Legacy mais actif pour les avis)
API_REVIEWS_BASE = "https://mybusiness.googleapis.com/v4"

OUTPUT_DIR = "data/gbp_reviews"
PARQUET_FILE = f"{OUTPUT_DIR}/reviews_all_locations.parquet"

os.makedirs(OUTPUT_DIR, exist_ok=True)

def get_access_token() -> str:
    """Récupère un access token via le refresh token."""
    payload = {
        "client_id": CLIENT_ID,
        "client_secret": CLIENT_SECRET,
        "refresh_token": REFRESH_TOKEN,
        "grant_type": "refresh_token",
    }

    try:
        response = requests.post(TOKEN_URL, data=payload)
        response.raise_for_status()
        return response.json()["access_token"]
    except requests.exceptions.HTTPError as e:
        print(f"Erreur d'authentification : {e.response.text}")
        raise

# ============================================================
# LOGIQUE DE COLLECTE (MULTI-API)
# ============================================================

def api_request(url: str, headers: Dict, method: str = "GET", params: Optional[Dict] = None) -> Dict:
    """Requête générique avec backoff TRÈS robuste (délais longs)."""
    max_retries = 10

    for attempt in range(max_retries):
        try:
            response = requests.get(url, headers=headers, params=params) if method == "GET" else None

            # Gestion du quota (429)
            if response.status_code == 429:
                # Stratégie agressive : on commence à 10s et on double
                # 10s, 20s, 40s, 80s, 160s...
                wait_time = 10 * (2 ** attempt)
                print(f"  ⚠️ Quota atteint (429). Tentative {attempt+1}/{max_retries}. Pause de {wait_time}s...")
                time.sleep(wait_time)
                continue

            # Gestion des erreurs serveur temporaires (500, 502, 503)
            if 500 <= response.status_code < 600:
                print(f"  ⚠️ Erreur serveur Google ({response.status_code}). Pause de 5s...")
                time.sleep(5)
                continue

            response.raise_for_status()
            return response.json()

        except requests.exceptions.RequestException as e:
            # Erreurs réseau (DNS, connexion coupée, etc.)
            print(f"  ⚠️ Erreur réseau : {e}. Pause de 5s...")
            if attempt == max_retries - 1:
                print(f"❌ Échec définitif sur : {url}")
                raise
            time.sleep(5)

    return {}

def test_api_connection(headers: Dict) -> bool:
    """
    Effectue un test simple de connectivité sur l'endpoint Accounts.
    Permet de valider le Token et les quotas avant de lancer le gros traitement.
    """
    print("\n--- 🧪 TEST DE CONNEXION INITIAL (Sanity Check) ---")
    url = f"{API_ACCOUNTS_BASE}/accounts"
    try:
        print(f"Tentative de contact : {url}")
        response = requests.get(url, headers=headers)

        print(f"Code retour HTTP : {response.status_code}")

        if response.status_code == 200:
            data = response.json()
            acc_count = len(data.get("accounts", []))
            print(f"✅ SUCCÈS : Connexion établie. {acc_count} compte(s) trouvé(s).")
            return True
        elif response.status_code == 429:
            print("❌ ÉCHEC : Quota dépassé dès le test (429). Attendez quelques minutes.")
            return False
        else:
            print(f"❌ ÉCHEC : Erreur API ({response.text})")
            return False

    except Exception as e:
        print(f"❌ ÉCHEC TECHNIQUE : {str(e)}")
        return False

def get_accounts(headers: Dict) -> List[Dict]:
    """Utilise l'API Account Management v1."""
    url = f"{API_ACCOUNTS_BASE}/accounts"
    data = api_request(url, headers)
    return data.get("accounts", [])

def get_locations(account_name: str, headers: Dict) -> List[Dict]:
    """
    Utilise l'API Business Information v1.
    IMPORTANT : Le readMask est OBLIGATOIRE pour cette API.
    """
    url = f"{API_LOCATIONS_BASE}/{account_name}/locations"
    # On demande explicitement les champs nécessaires
    params = {"readMask": "name,title,storeCode", "pageSize": 100}

    locations = []
    while True:
        data = api_request(url, headers, params=params)
        locations.extend(data.get("locations", []))

        next_token = data.get("nextPageToken")
        if next_token:
            params["pageToken"] = next_token
        else:
            break

        # Petite pause pour éviter de spammer l'API lors de la pagination
        time.sleep(1.0)

    return locations

def get_reviews(formatted_location_name: str, headers: Dict) -> List[Dict]:
    """
    Utilise l'API v4 (Legacy) pour les avis.
    formatted_location_name doit être : accounts/{accId}/locations/{locId}
    """
    all_reviews = []
    # Endpoint v4
    url = f"{API_REVIEWS_BASE}/{formatted_location_name}/reviews"
    params = {"pageSize": 50}

    while True:
        try:
            data = api_request(url, headers, params=params)
        except Exception:
            # Si un établissement n'a pas d'avis ou erreur spécifique, on skip sans crasher tout le pipeline
            print(f"    ⚠️ Impossible de récupérer les avis pour {formatted_location_name} (ou aucun avis).")
            break

        batch = data.get("reviews", [])
        all_reviews.extend(batch)

        # Log métriques si disponibles
        if "totalReviewCount" in data and not all_reviews:
             print(f"    ℹ️ {data.get('totalReviewCount')} avis détectés (Moyenne: {data.get('averageRating')})")

        next_token = data.get("nextPageToken")
        if next_token:
            params["pageToken"] = next_token
        else:
            break

        # Pause de politesse entre les pages d'avis
        time.sleep(1.0)

    return all_reviews

# ============================================================
# NORMALISATION
# ============================================================

RATING_MAP = {
    "ONE": 1, "TWO": 2, "THREE": 3, "FOUR": 4, "FIVE": 5, "STAR_RATING_UNSPECIFIED": 0
}

def normalize_review(review: Dict, location: Dict, account_id: str) -> Dict:
    reviewer = review.get("reviewer", {})
    reply = review.get("reviewReply", {})
    raw_rating = review.get("starRating", "STAR_RATING_UNSPECIFIED")

    # Dans l'API v1, le nom de l'établissement est 'title', pas 'locationName'
    loc_name = location.get("title") or location.get("locationName")

    return {
        "account_id": account_id,
        "internal_location_name": loc_name,
        "store_code": location.get("storeCode"),
        "review_resource_name": review.get("name"),
        "review_id": review.get("reviewId"),
        "star_rating_label": raw_rating,
        "rating_value": RATING_MAP.get(raw_rating, 0),
        "comment": review.get("comment", ""),
        "create_time": review.get("createTime"),
        "update_time": review.get("updateTime"),
        "reviewer_name": reviewer.get("displayName"),
        "reviewer_photo": reviewer.get("profilePhotoUrl"),
        "is_anonymous": reviewer.get("isAnonymous", False),
        "reply_text": reply.get("comment"),
        "reply_date": reply.get("updateTime"),
        "ingestion_utc": datetime.utcnow().isoformat()
    }

# ============================================================
# EXÉCUTION
# ============================================================

def run_pipeline():
    print(f"--- Pipeline démarré à {datetime.now()} (Nouvelle Architecture v1/v4) ---")

    try:
        token = get_access_token()
        headers = {"Authorization": f"Bearer {token}"}

        # 1. Test de connexion préalable
        if not test_api_connection(headers):
            print("❌ Arrêt du pipeline suite à l'échec du test de connexion.")
            return

        dataset = []
        accounts = get_accounts(headers)
        print(f"\nComptes trouvés : {len(accounts)}")

        for acc in accounts:
            # acc['name'] est de la forme 'accounts/12345'
            acc_name = acc.get("name")
            print(f"Traitement du compte : {acc.get('accountName')} ({acc_name})")

            locations = get_locations(acc_name, headers)
            print(f"  > {len(locations)} établissements trouvés.")

            for i, loc in enumerate(locations):
                # loc['name'] API v1 est souvent 'locations/98765' (sans le préfixe account)
                # L'API v4 Reviews a besoin de 'accounts/12345/locations/98765'
                loc_id_part = loc.get("name").split("/")[-1]
                full_resource_name = f"{acc_name}/locations/{loc_id_part}"

                print(f"  > [{i+1}/{len(locations)}] Récupération avis : {loc.get('title')}...")

                reviews = get_reviews(full_resource_name, headers)
                for r in reviews:
                    dataset.append(normalize_review(r, loc, acc_name))

                # IMPORTANT : Pause pour éviter l'erreur 429 entre deux établissements
                time.sleep(2.0)

        if dataset:
            df = pd.DataFrame(dataset)
            for col in ["create_time", "update_time", "reply_date"]:
                df[col] = pd.to_datetime(df[col], errors="coerce")

            df.to_parquet(PARQUET_FILE, engine="pyarrow", index=False)
            print(f"\n✅ SUCCÈS : {len(df)} avis exportés dans {PARQUET_FILE}")
            print(f"Note moyenne : {df['rating_value'].mean():.2f}")
        else:
            print("\n⚠️ Aucun avis trouvé ou erreur de collecte.")

    except Exception as e:
        print(f"\n❌ ERREUR CRITIQUE PIPELINE : {str(e)}")

if __name__ == "__main__":
    run_pipeline()

--- Pipeline démarré à 2026-01-30 03:11:47.365416 (Nouvelle Architecture v1/v4) ---

--- 🧪 TEST DE CONNEXION INITIAL (Sanity Check) ---
Tentative de contact : https://mybusinessaccountmanagement.googleapis.com/v1/accounts
Code retour HTTP : 429
❌ ÉCHEC : Quota dépassé dès le test (429). Attendez quelques minutes.
❌ Arrêt du pipeline suite à l'échec du test de connexion.
